<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/DAPT_TAPT_unipelt_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHANGE AFTER Test Run
*   CHANGE NAME
*   PUT EPOCHS BACK AT 100



In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
# !pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-n

In [4]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [5]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

dataset = load_dataset(dataset_name)

print(dataset)

Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 44876
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9754
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1980
    })
})


In [6]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import UniPELTConfig

adapter_config = UniPELTConfig()
adapter_name = "dapt_tapt_unipelt"
adapter_hub_name = f"TEST_pretrain_{adapter_name}_adapter"

In [8]:
print(adapter_hub_name)

TEST_pretrain_dapt_tapt_unipelt_adapter


In [9]:
dapt_adapter_name = "jgrc3/RobertaDAPT_adapters_unipelt"

In [10]:
# Add a new adapter
loaded_name = model.load_adapter(dapt_adapter_name, config=adapter_config)

# Set the adapter to be used for training
model.train_adapter(loaded_name)

print(f"Loaded Adapter Name: {loaded_name}")

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

Loaded Adapter Name: review_adapter_unipelt


In [11]:
summary = model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
review_adapter_unipelt   union            11,083,376       8.892       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [14]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="dapt_tapt_unipelt_helpfulness_pretrained_model",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=1,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [15]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [16]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=pretraining_args,
    train_dataset=dataset["test"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [17]:
pretrain_results = trainer.train()
print(pretrain_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,1.724400,1.584900


TrainOutput(global_step=232, training_loss=1.7244129838614628, metrics={'train_runtime': 275.4207, 'train_samples_per_second': 35.415, 'train_steps_per_second': 0.842, 'total_flos': 2914472299511808.0, 'train_loss': 1.7244129838614628, 'epoch': 1.0})


In [18]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

{'eval_loss': 1.5827726125717163, 'eval_runtime': 116.2491, 'eval_samples_per_second': 83.906, 'eval_steps_per_second': 4.0, 'epoch': 1.0}


In [19]:
model.push_adapter_to_hub(
    adapter_hub_name,
    loaded_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/TEST_pretrain_dapt_tapt_unipelt_adapter/commit/d06ec94cdd1efbccab12c5d8402881b9bfb9ffca', commit_message='Upload model', commit_description='', oid='d06ec94cdd1efbccab12c5d8402881b9bfb9ffca', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1713532690.fa5501221c58.195.0:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1713533083.fa5501221c58.195.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/dapt_tapt_unipelt_helpfulness_pretrained_model/commit/e2048ccc286f7e500c8cbe8505f083b0a0f9a747', commit_message='End of training', commit_description='', oid='e2048ccc286f7e500c8cbe8505f083b0a0f9a747', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()